<a href="https://colab.research.google.com/github/ldeluigi/supermarket-2077-product-vision/blob/master/ProductDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Detection

## Download datasets

In [ ]:
!rm -rf sample_data
!gdown --id 1fDr4g4wbnSRkuCYyS3wpuJS7Ax22bVB_ -O all.zip
!unzip -oq all.zip

%matplotlib inline

## Imports

In [ ]:
import scipy.io
import os
import re
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

## Read store data

In [ ]:
def read_store_data(storename):
  dirname_anno = os.path.join(storename, 'annotation')
  dirname_images = os.path.join(storename, 'images')
  directory_anno = os.fsencode(dirname_anno)
  directory_images = os.fsencode(dirname_images)

  result = []

  for file in os.listdir(directory_anno):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"): 
      mat = scipy.io.loadmat(os.path.join(dirname_anno, filename))
      number = re.search(r'^anno.(\d+).mat$', filename).group(1)
      img = cv2.imread(os.path.join(dirname_images, number + '.jpg'))

      img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img_annotation = mat['annotation'][0, 0]
      bboxes = map(lambda x: x[0], img_annotation[0][0])
      labels = map(lambda x: str(x[0][0][0]), img_annotation[1][0])
      class_indexes = img_annotation[2][0]
      result.append((img_rgb, list(zip(bboxes, labels, class_indexes))))
  return np.rec.array(result, dtype=[('image', 'O'), ('items', 'O')])

## Read training data

In [ ]:
dirname = 'Training'

def read_classes():
  mat = scipy.io.loadmat(os.path.join(dirname, 'TrainingClassesIndex.mat'))
  classes = list(map(lambda x: x[0], mat['classes'][0]))
  return dict(enumerate(classes, start=1))

def read_training_data(classes):
  result = []
  for class_index, class_name in classes.items():
    dirname_images = os.path.join(dirname, class_name)
    directory_images = os.fsencode(dirname_images)
    for file in os.listdir(directory_images):
      img = cv2.imread(os.path.join(dirname_images, os.fsdecode(file)))
      img_rgb =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      result.append((img_rgb, class_index))
  return np.rec.array(result, dtype=[('image', 'O'), ('class_index', 'i4')])

## Prepare products dataset

In [ ]:
classes = read_classes()
products = read_training_data(classes)
store = read_store_data('store1')

def class_name(class_index):
  return classes[class_index] if class_index >= 0 else None

In [ ]:
def summary_of_dataset(dataset):
  print('Information on class occurencies:')
  c = np.vectorize(class_name)(dataset['class_index'])
  print(c)
  # TODO
  print('\n\nInformation on image byte occupation:')
  info = np.vectorize(lambda x: np.prod(x.shape))(dataset['image'])
  print(info)
  # TODO

summary_of_dataset(products)

## Data visualization

In [ ]:
def show_image(img):
  plt.axis('off')
  plt.imshow(img)

def show_grayscale_image(img):
  show_image(cv2.merge([img, img, img]))

In [ ]:
def plot_grid(images, columns, show_axis=False):
  fig = plt.figure(figsize=(12, 6), dpi=120)
  for index, img in enumerate(images, start=1):
    sp = fig.add_subplot(math.ceil(len(images) / columns), columns, index)
    if not show_axis:
      plt.axis('off')
    plt.imshow(img)
    sp.set_title(index, fontsize=10)

def dataset_plot_grid(indexes, columns, dataset, draw_item):
  fig = plt.figure(figsize=(12, 6), dpi=120)
  # fig.subplots_adjust(hspace=0.2)
  for index, i_img in enumerate(indexes, start=1):
    sp = fig.add_subplot(math.ceil(len(indexes) / columns), columns, index)
    row = dataset[i_img]
    draw_item(row, sp)

In [ ]:
def show_products_with_class(indexes, columns, dataset):
  def show_single_product_with_class(row, sp):
    plt.axis('off')
    plt.imshow(row.image)
    sp.set_title(class_name(row.class_index), fontsize=10)
  dataset_plot_grid(indexes, columns, dataset, show_single_product_with_class)

show_products_with_class(np.random.randint(0, len(products), 6), 2, products)

In [ ]:
def show_shelf_with_bbox_and_classes(indexes, columns, dataset):
  def show_single_shelf(row, sp):
    color = (255, 255, 0)
    img = row.image.copy()
    for item in row.items:
      bbox, label, class_index  = item
      [xmin, xmax, ymin, ymax] = bbox
      height, width, _ = img.shape
      xmin = int(xmin * width)
      xmax = int(xmax * width)
      ymin = int(ymin * height)
      ymax = int(ymax * height)
      thickness = round(max(width, height) * 0.01)
      cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, thickness)
      cv2.putText(img,
                  class_name(class_index),
                  (round(xmin + thickness * 1.5), round(ymin + thickness * 3)),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  2.5,
                  color,
                  thickness=round(thickness/2))
    plt.axis('off')
    plt.imshow(img)
  dataset_plot_grid(indexes, columns, dataset, show_single_shelf)

show_shelf_with_bbox_and_classes(np.random.randint(0, len(store), 4), 2, store)

## Data manipulation

### Background removal

In [ ]:
# code taken from https://www.kaggle.com/vadbeg/opencv-background-removal and modified

def remove_background(img, threshold):
    """
    This method removes background from your image
    
    :param img: cv2 image
    :type img: np.array
    :param threshold: threshold value for cv2.threshold
    :type threshold: float
    :return: RGBA image
    :rtype: np.ndarray
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, threshed = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)

    kernel_size = round(max(img.shape[0], img.shape[1]) * 0.02)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(morphed, 
                            cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[0]

    cnts = sorted(cnts, key=cv2.contourArea)

    mask = cv2.drawContours(threshed, [cnts[-1]], 0, [255], cv2.FILLED)
    masked_data = cv2.bitwise_and(img, img, mask=mask)

    # for i in range(img.shape[0]):
    #   for j in range(img.shape[1]):
    #     masked_data[i, j] = img[i, j].copy() if mask[i, j] == 255 else 0

    x, y, w, h = cv2.boundingRect(cnts[-1])
    dst = masked_data[y: y + h, x: x + w]

    alpha = mask[y: y + h, x: x + w]
    r, g, b = cv2.split(dst)

    rgba = [r, g, b, alpha]
    dst = cv2.merge(rgba, 4)
    return dst

n = np.random.randint(products.shape[0])
print(f'Index: {n}')
print(f'Class: {class_name(products[n].class_index)}')
plot_grid([products[n].image, remove_background(products[n].image, 250)], 2, show_axis=True)

### Image resize

In [ ]:
def resize_image(img, size, color=[0,0,0,0]):
  target_w, target_h = size
  original_h, original_w, _ = img.shape
  target_ar = target_w / target_h
  original_ar = original_w / original_h

  scale_factor = target_h / original_h if target_ar > original_ar else target_w / original_w
  scaled_w = round(original_w * scale_factor)
  scaled_h = round(original_h * scale_factor)
  scaled_size = (scaled_w, scaled_h)
  resized = cv2.resize(img, scaled_size)

  delta_h = target_h - scaled_h
  delta_w = target_w - scaled_w
  top, bottom = delta_h // 2, delta_h - (delta_h // 2)
  left, right = delta_w // 2, delta_w - (delta_w // 2)

  return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

n = np.random.randint(products.shape[0])
#n = 1000
image = products[n].image
image = remove_background(image, 250)
plot_grid([image, resize_image(image, (400, 400))], 2, show_axis=True)

### Clean dataset

In [ ]:
size = (256, 256)

def clean_image(img):
  threshold = 250
  img = remove_background(img, threshold)
  return resize_image(img, size)

train_x = np.array(list(map(lambda x: clean_image(x.image), products)))
print(train_x.shape)
print(train_x.dtype)